# Benders' Decomposition


Task: Solve the Computational Example 3.1 from the textbook "Decomposition Techniques in Mathematical Programming" using Benders' decomposition.

In [1]:
using JuMP
using GLPK

In [2]:
# Define the problem data
c1 = -1/4  # Coefficients for the x variable in the objective function
c2 = -1    # Coefficients for the y variable in the subproblem
α_down = -25 # Initial constraint for α

-25

In [3]:
comp_example = Model()
@variable(comp_example, 0 <= x <= 16)
@variable(comp_example, y >= 0)
@constraint(comp_example, y - x <= 5)
@constraint(comp_example, y - 0.5 * x <= 15/2)
@constraint(comp_example, y + 0.5 * x <= 35/2)
@constraint(comp_example, -y + x <= 10)
@objective(comp_example, Min, c2 * y + c1 * x )

print(comp_example)

Here, we can see the example from the textbook.

## Model Definition

In [4]:
master_model = Model(GLPK.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

In [5]:
# Master problem
@variable(master_model, 0 <= x <= 16)
@variable(master_model, α >= α_down)
@objective(master_model, Min, c1 * x + α)

print(master_model)

In [6]:
# Defines and solves subproblem using updated x value
function solve_subproblem(x_val)
    println("Value of x: $x_val")
    
    sub_model = Model(GLPK.Optimizer)
    @variable(sub_model, y >= 0)
    
    # Define constraints 
    @constraint(sub_model, con1, y - x_val <= 5)
    @constraint(sub_model, con2, y - 0.5 * x_val <= 15/2)
    @constraint(sub_model, con3, y + 0.5 * x_val <= 35/2)
    @constraint(sub_model, con4, -y + x_val <= 10)
    
    # Define the objective function
    @objective(sub_model, Min, c2 * y)
    println("Subproblem:")
    print(sub_model)
    optimize!(sub_model)
    
    # Ensure that the subproblem was solved to optimality
    if termination_status(sub_model) != MOI.OPTIMAL
        error("Subproblem did not solve optimally.")
    end
    
    # Retrieve the optimal value and the dual values
    obj_value = objective_value(sub_model)
    duals = [dual(con1), dual(con2), dual(con3), dual(con4)]
    y_value = value(y)
    
    return (obj = obj_value, y = y_value, λ = duals)
end


solve_subproblem (generic function with 1 method)

## Benders' Loop

In [7]:
# Optimization loop
max_iterations = 20
ε = 1e-20


1.0e-20

In [8]:
# Benders' decomposition loop
y_val = 0
for k in 1:max_iterations
    optimize!(master_model)
    
    if termination_status(master_model) != MOI.OPTIMAL
        error("Master problem did not solve optimally at iteration $k")
    end

    lower_bound = objective_value(master_model)
    x_val = value(x)  # Update x value
    subproblem_solution = solve_subproblem(x_val)  # Solve subproblem using new x value
    println("Subproblem objective at iteration $k: ", subproblem_solution.obj)
    println("Dual variables at iteration $k: ", subproblem_solution.λ)
    y_val = subproblem_solution.y

    upper_bound = c1 * x_val + subproblem_solution.obj
    gap = upper_bound - lower_bound
    println("Iteration $k: Lower Bound = $lower_bound, Upper Bound = $upper_bound, Gap = $gap")

    # Check for convergence
    if gap < ε
        println("Terminating with the optimal solution")
        break
    end


    # Add Benders' cut to master problem
    @constraint(master_model, α >= subproblem_solution.λ[1] * (5 + x) +
                  subproblem_solution.λ[2] * (15/2 + 1/2 * x) +
                  subproblem_solution.λ[3] * (35/2 - 1/2 * x) +
                  subproblem_solution.λ[4] * (10 - x))
    println("Master Problem:")
    print(master_model)
end

# Output optimal values
println("Optimal x: ", value(x))
println("Optimal α: ", value(α))
println("Optimal y: ", y_val)

Value of x: 16.0
Subproblem:


Subproblem objective at iteration 1: -9.5
Dual variables at iteration 1: [0.0, 0.0, -1.0, 0.0]
Iteration 1: Lower Bound = -29.0, Upper Bound = -13.5, Gap = 15.5
Master Problem:


Value of x: 0.0
Subproblem:


Subproblem objective at iteration 2: -5.0
Dual variables at iteration 2: [-1.0, 0.0, 0.0, 0.0]
Iteration 2: Lower Bound = -17.5, Upper Bound = -5.0, Gap = 12.5
Master Problem:


Value of x: 8.333333333333334
Subproblem:


Subproblem objective at iteration 3: -11.666666666666668
Dual variables at iteration 3: [0.0, -1.0, 0.0, 0.0]
Iteration 3: Lower Bound = -15.416666666666666, Upper Bound = -13.750000000000002, Gap = 1.6666666666666643
Master Problem:


Value of x: 9.999999999999998
Subproblem:


Subproblem objective at iteration 4: -12.5
Dual variables at iteration 4: [0.0, -1.0, 0.0, 0.0]
Iteration 4: Lower Bound = -14.999999999999998, Upper Bound = -15.0, Gap = -1.7763568394002505e-15
Terminating with the optimal solution
Optimal x: 9.999999999999998
Optimal α: -12.499999999999998
Optimal y: 12.5


We can see that the optimal values for x, α, and y are equal to those from the textbook, when rounded correctly. I am assuming that the small rounding error stems from the solver.